# LSTM MODEL :  미래 유동인구 예측
---

In [2]:
# 모듈 로딩
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import numpy as np
import matplotlib.pyplot as p


import random
random.seed(42)

import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')

## [1] 데이터 전처리
---

## [2] 모델 생성
---


데이터 과거단위 생성 함수

In [3]:
# [ 학습 단위 생성 함수 ]
# data = 사용할 데이터
# n = 학습할 과거 데이터 수
# --> 예측값 개수 : 전체 데이터 갯수 - n (앞쪽데이터 n개가 제외됨)

def generateX(data, n): # data, n=5(예시)
    x_train = []
    y_train = []
    for i in range(len(data)): # i = 0~9 
        x = data[i:(i + n)] # x : 0~4,1~5,2~6,3~7,4~8,5~9...9~14
        if (i + n) < len(data): # target data가 있는 범위까지만
            x_train.append(x) # x_train = 0~4,1~5,2~6,3~7,4~8까지
            y_train.append(data[i + n]) # y_train = 5,6,7,8,9
        else:
            break
    return np.array(x_train), np.array(y_train)
    # x_train 개수: date개수 - n(뒤)
    # y_train 개수: date개수 - n(앞)

In [4]:
def pred_future(DATA,WINDOW_SIZE,TEST_SIZE):
    
    print(f'예측 개월 수: {WINDOW_SIZE}개월')
    print('--------------------------------')

    # [ 과거데이터 학습 단위 설정]
    x, y = generateX(DATA, WINDOW_SIZE) 

    print('[ before reshape ]')
    print('x.shape:',x.shape)
    print('y.shape:',y.shape)
    print('--------------------------------')
    
    # 차원 하나씩 추가 -----------------------------------------------------
    x = x.reshape(-1,WINDOW_SIZE,1)
    y = y.reshape(-1,1)

    print('[ after reshape ]')
    print('x.shape:', x.shape)
    print('y.shape:', y.shape)
    print('--------------------------------')

    # [ 학습 사이즈 ] -> train, test 
    # train_size = len(DATA)-(2*WINDOW_SIZE)

    # 학습용 데이터와 시험용 데이터
    x_train = x[:-TEST_SIZE, :, :]
    y_train = y[:-TEST_SIZE, :]
    x_test = x[TEST_SIZE:, :, :]
    y_test = y[TEST_SIZE:, :]

    print('x_train.shape:', x_train.shape,'y_train.shape:', y_train.shape)
    print('x_test.shape :', x_test.shape, 'y_test.shape :', y_test.shape)
    print('--------------------------------')

    # 모델 구성 ---------------------------------------------------------------
    model = Sequential()
    model.add(LSTM(16, 
                input_shape=(x_train.shape[1], x_train.shape[2]), 
                activation='relu', 
                return_sequences=False)
            )

    model.add(Dense(1))
    
    # 모델 컴파일 --------------------------------------------------------------
    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stop = EarlyStopping(monitor='loss', patience=10)

    history = model.fit(x_train, y_train, 
                                    epochs=10, 
                                    batch_size=16)
                                    # callbacks=[early_stop])
    
    y_hat = model.predict(x_test)

    # 예측 결과 시각화 ----------------------------------------------------------
    a_axis = np.arange(0, len(y_train))
    b_axis = np.arange(len(y_train), len(y_train) + len(y_hat))

    plt.figure(figsize=(16,9)) 
    plt.plot(a_axis, y_train.reshape(x_train.shape[0],), 'o-') 
    plt.plot(b_axis, y_hat.reshape(x_test.shape[0],), 'o-', color='red', label='Predicted')
    plt.plot(b_axis, y_test.reshape(x_test.shape[0],), 'o-', color='green', alpha=0.2, label='Actual')
    plt.legend()
    plt.show()

    return y_hat

In [5]:
# 미래 예측 함수 테스트(sin 그래프)

# [ 조정 변수]

# 총 데이터
# DATA = np.sin(2 * np.pi * 0.03 * np.arange(0, 300)) + np.random.random(300) # Sine 함수에 노이즈를 섞은 데이터로 학습 데이터 100개를 생성한다
df = pd.read_csv("../data/data.csv")
DATA = df["values"]

# 과거 학습 단위 = 예측하게 되는 타겟 갯수
WINDOW_SIZE = 7

# 테스트 사이즈 설정
TEST_SIZE = 2500

# past_learn: 과거 학습할 데이터 단위 
pred = pred_future(DATA,WINDOW_SIZE,TEST_SIZE)

예측 개월 수: 7개월
--------------------------------
[ before reshape ]
x.shape: (8753, 7)
y.shape: (8753,)
--------------------------------
[ after reshape ]
x.shape: (8753, 7, 1)
y.shape: (8753, 1)
--------------------------------
x_train.shape: (6253, 7, 1) y_train.shape: (6253, 1)
x_test.shape : (6253, 7, 1) y_test.shape : (6253, 1)
--------------------------------
Epoch 1/10
391/391 [==============================] - 2s 2ms/step - loss: 12423369.0000
Epoch 2/10
391/391 [==============================] - 1s 2ms/step - loss: 13449304.0000
Epoch 3/10
391/391 [==============================] - 1s 2ms/step - loss: 11553738.0000
Epoch 4/10
391/391 [==============================] - 1s 2ms/step - loss: 10800374.0000
Epoch 5/10
391/391 [==============================] - 1s 2ms/step - loss: 10432779.0000
Epoch 6/10
391/391 [==============================] - 1s 2ms/step - loss: 9791427.0000
Epoch 7/10
391/391 [==============================] - 1s 2ms/step - loss: 9078592.0000
Epoch 8/10
391/391 [=

: 

: 